In [112]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from scipy import stats
# import tensorflow as tf

In [113]:
# Import our mental health dataset and create a dataframe
mental_df = pd.read_csv('osmi 2016.csv')
mental_df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,25-Jun,1.0,NaN,No,Yes,Yes,Yes,Yes,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,25-Jun,1.0,NaN,No,NaN,No,No,I don't know,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Sometimes,43,male,United Kingdom,NaN,United Kingdom,NaN,Supervisor/Team Lead,Sometimes
4,0,25-Jun,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Not applicable to me,Not applicable to me,34,Female,United States of America,New York,United States of America,New York,Other,Sometimes
1429,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Often,56,MALE,United States of America,California,Afghanistan,NaN,Support,Sometimes
1430,0,100-500,1.0,NaN,Yes,Yes,Yes,Yes,I don't know,Somewhat difficult,...,Rarely,Sometimes,52,Male,United States of America,Georgia,United States of America,Georgia,Back-end Developer,Sometimes
1431,0,100-500,0.0,1.0,I don't know,I am not sure,No,Yes,I don't know,Somewhat difficult,...,Sometimes,Often,30,Female,United States of America,Nebraska,United States of America,Nebraska,DevOps/SysAdmin,Sometimes


In [114]:
# Replace all NaN values with 0
mental_df = mental_df.replace(to_replace=pd.np.nan, value=0)

/tmp/ipykernel_1751/1301895068.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  mental_df = mental_df.replace(to_replace=pd.np.nan, value=0)


In [115]:
mental_df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,0.0,Not eligible for coverage / N/A,0,No,No,I don't know,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,0,United Kingdom,0,Back-end Developer,Sometimes
1,0,25-Jun,1.0,0.0,No,Yes,Yes,Yes,Yes,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,25-Jun,1.0,0.0,No,0,No,No,I don't know,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,0,United Kingdom,0,Back-end Developer,Always
3,1,0,0.0,0.0,0,0,0,0,0,0,...,Sometimes,Sometimes,43,male,United Kingdom,0,United Kingdom,0,Supervisor/Team Lead,Sometimes
4,0,25-Jun,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,0,0.0,0.0,0,0,0,0,0,0,...,Not applicable to me,Not applicable to me,34,Female,United States of America,New York,United States of America,New York,Other,Sometimes
1429,1,0,0.0,0.0,0,0,0,0,0,0,...,Sometimes,Often,56,MALE,United States of America,California,Afghanistan,0,Support,Sometimes
1430,0,100-500,1.0,0.0,Yes,Yes,Yes,Yes,I don't know,Somewhat difficult,...,Rarely,Sometimes,52,Male,United States of America,Georgia,United States of America,Georgia,Back-end Developer,Sometimes
1431,0,100-500,0.0,1.0,I don't know,I am not sure,No,Yes,I don't know,Somewhat difficult,...,Sometimes,Often,30,Female,United States of America,Nebraska,United States of America,Nebraska,DevOps/SysAdmin,Sometimes


In [116]:
mental_df = mental_df.apply(lambda x: x.astype(str).str.lower())

In [117]:
mental_df.shape

(1433, 63)

In [118]:
mental_df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,0.0,not eligible for coverage / n/a,0,no,no,i don't know,very easy,...,not applicable to me,not applicable to me,39,male,united kingdom,0,united kingdom,0,back-end developer,sometimes
1,0,25-jun,1.0,0.0,no,yes,yes,yes,yes,somewhat easy,...,rarely,sometimes,29,male,united states of america,illinois,united states of america,illinois,back-end developer|front-end developer,never
2,0,25-jun,1.0,0.0,no,0,no,no,i don't know,neither easy nor difficult,...,not applicable to me,not applicable to me,38,male,united kingdom,0,united kingdom,0,back-end developer,always
3,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,sometimes,43,male,united kingdom,0,united kingdom,0,supervisor/team lead,sometimes
4,0,25-jun,0.0,1.0,yes,yes,no,no,no,neither easy nor difficult,...,sometimes,sometimes,43,female,united states of america,illinois,united states of america,illinois,executive leadership|supervisor/team lead|dev ...,sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,0,0.0,0.0,0,0,0,0,0,0,...,not applicable to me,not applicable to me,34,female,united states of america,new york,united states of america,new york,other,sometimes
1429,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,often,56,male,united states of america,california,afghanistan,0,support,sometimes
1430,0,100-500,1.0,0.0,yes,yes,yes,yes,i don't know,somewhat difficult,...,rarely,sometimes,52,male,united states of america,georgia,united states of america,georgia,back-end developer,sometimes
1431,0,100-500,0.0,1.0,i don't know,i am not sure,no,yes,i don't know,somewhat difficult,...,sometimes,often,30,female,united states of america,nebraska,united states of america,nebraska,devops/sysadmin,sometimes


In [119]:
# Print out gender counts
gender_counts = mental_df['What is your gender?'].value_counts()
gender_counts

male                                                                                                                                                             860
female                                                                                                                                                           248
m                                                                                                                                                                165
f                                                                                                                                                                 61
male                                                                                                                                                              13
female                                                                                                                                                            12
woman     

In [120]:
# Determine which values to replace
replace_genders = list(gender_counts[gender_counts < 20].index)

# Replace in DataFrame
for gender in replace_genders:
    mental_df['What is your gender?'] = mental_df['What is your gender?'].replace(gender,"Other")

In [121]:
# Print out gender counts
gender_counts = mental_df['What is your gender?'].value_counts()
gender_counts

male      860
female    248
m         165
Other      99
f          61
Name: What is your gender?, dtype: int64

In [122]:
# Print out country counts
country_counts = mental_df['What country do you live in?'].value_counts()
country_counts

united states of america    840
united kingdom              180
canada                       78
germany                      58
netherlands                  48
australia                    35
sweden                       19
france                       16
ireland                      15
switzerland                  10
brazil                       10
new zealand                   9
india                         9
russia                        9
bulgaria                      7
finland                       7
denmark                       7
italy                         5
belgium                       5
south africa                  4
romania                       4
spain                         4
poland                        4
austria                       4
norway                        3
pakistan                      3
chile                         3
czech republic                3
japan                         2
israel                        2
other                         2
bosnia a

In [123]:
# Determine which values to replace
replace_countries = list(country_counts[country_counts < 20].index)

# Replace in DataFrame
for country in replace_countries:
    mental_df['What country do you live in?'] = mental_df['What country do you live in?'].replace(country,"Other")

In [124]:
# Print out country counts
country_counts = mental_df['What country do you live in?'].value_counts()
country_counts

united states of america    840
Other                       194
united kingdom              180
canada                       78
germany                      58
netherlands                  48
australia                    35
Name: What country do you live in?, dtype: int64

In [125]:
# Print out country counts
country_counts = mental_df['What country do you work in?'].value_counts()
country_counts

united states of america    851
united kingdom              183
canada                       74
germany                      58
netherlands                  47
australia                    34
sweden                       20
ireland                      15
france                       14
switzerland                  10
brazil                       10
india                         9
russia                        9
new zealand                   9
denmark                       7
bulgaria                      7
finland                       7
belgium                       5
south africa                  4
poland                        4
austria                       4
czech republic                3
italy                         3
chile                         3
norway                        3
romania                       3
spain                         3
other                         2
bosnia and herzegovina        2
afghanistan                   2
pakistan                      2
israel  

In [126]:
# Determine which values to replace
replace_countries = list(country_counts[country_counts < 20].index)

# Replace in DataFrame
for country in replace_countries:
    mental_df['What country do you work in?'] = mental_df['What country do you work in?'].replace(country,"Other")

In [127]:
mental_df


,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,0.0,not eligible for coverage / n/a,0,no,no,i don't know,very easy,...,not applicable to me,not applicable to me,39,male,united kingdom,0,united kingdom,0,back-end developer,sometimes
1,0,25-jun,1.0,0.0,no,yes,yes,yes,yes,somewhat easy,...,rarely,sometimes,29,male,united states of america,illinois,united states of america,illinois,back-end developer|front-end developer,never
2,0,25-jun,1.0,0.0,no,0,no,no,i don't know,neither easy nor difficult,...,not applicable to me,not applicable to me,38,Other,united kingdom,0,united kingdom,0,back-end developer,always
3,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,sometimes,43,male,united kingdom,0,united kingdom,0,supervisor/team lead,sometimes
4,0,25-jun,0.0,1.0,yes,yes,no,no,no,neither easy nor difficult,...,sometimes,sometimes,43,female,united states of america,illinois,united states of america,illinois,executive leadership|supervisor/team lead|dev ...,sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,0,0.0,0.0,0,0,0,0,0,0,...,not applicable to me,not applicable to me,34,Other,united states of america,new york,united states of america,new york,other,sometimes
1429,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,often,56,male,united states of america,california,Other,0,support,sometimes
1430,0,100-500,1.0,0.0,yes,yes,yes,yes,i don't know,somewhat difficult,...,rarely,sometimes,52,male,united states of america,georgia,united states of america,georgia,back-end developer,sometimes
1431,0,100-500,0.0,1.0,i don't know,i am not sure,no,yes,i don't know,somewhat difficult,...,sometimes,often,30,female,united states of america,nebraska,united states of america,nebraska,devops/sysadmin,sometimes


In [128]:
mental_df.dropna(inplace=True)

In [129]:
mental_df.shape

(1433, 63)

In [130]:
mental_df.replace(",", "")

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,0.0,not eligible for coverage / n/a,0,no,no,i don't know,very easy,...,not applicable to me,not applicable to me,39,male,united kingdom,0,united kingdom,0,back-end developer,sometimes
1,0,25-jun,1.0,0.0,no,yes,yes,yes,yes,somewhat easy,...,rarely,sometimes,29,male,united states of america,illinois,united states of america,illinois,back-end developer|front-end developer,never
2,0,25-jun,1.0,0.0,no,0,no,no,i don't know,neither easy nor difficult,...,not applicable to me,not applicable to me,38,Other,united kingdom,0,united kingdom,0,back-end developer,always
3,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,sometimes,43,male,united kingdom,0,united kingdom,0,supervisor/team lead,sometimes
4,0,25-jun,0.0,1.0,yes,yes,no,no,no,neither easy nor difficult,...,sometimes,sometimes,43,female,united states of america,illinois,united states of america,illinois,executive leadership|supervisor/team lead|dev ...,sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,0,0.0,0.0,0,0,0,0,0,0,...,not applicable to me,not applicable to me,34,Other,united states of america,new york,united states of america,new york,other,sometimes
1429,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,often,56,male,united states of america,california,Other,0,support,sometimes
1430,0,100-500,1.0,0.0,yes,yes,yes,yes,i don't know,somewhat difficult,...,rarely,sometimes,52,male,united states of america,georgia,united states of america,georgia,back-end developer,sometimes
1431,0,100-500,0.0,1.0,i don't know,i am not sure,no,yes,i don't know,somewhat difficult,...,sometimes,often,30,female,united states of america,nebraska,united states of america,nebraska,devops/sysadmin,sometimes


In [131]:
mental_df.to_csv('cleaned_data.csv', index=False)

In [132]:
mental_df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,0.0,not eligible for coverage / n/a,0,no,no,i don't know,very easy,...,not applicable to me,not applicable to me,39,male,united kingdom,0,united kingdom,0,back-end developer,sometimes
1,0,25-jun,1.0,0.0,no,yes,yes,yes,yes,somewhat easy,...,rarely,sometimes,29,male,united states of america,illinois,united states of america,illinois,back-end developer|front-end developer,never
2,0,25-jun,1.0,0.0,no,0,no,no,i don't know,neither easy nor difficult,...,not applicable to me,not applicable to me,38,Other,united kingdom,0,united kingdom,0,back-end developer,always
3,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,sometimes,43,male,united kingdom,0,united kingdom,0,supervisor/team lead,sometimes
4,0,25-jun,0.0,1.0,yes,yes,no,no,no,neither easy nor difficult,...,sometimes,sometimes,43,female,united states of america,illinois,united states of america,illinois,executive leadership|supervisor/team lead|dev ...,sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,0,0.0,0.0,0,0,0,0,0,0,...,not applicable to me,not applicable to me,34,Other,united states of america,new york,united states of america,new york,other,sometimes
1429,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,often,56,male,united states of america,california,Other,0,support,sometimes
1430,0,100-500,1.0,0.0,yes,yes,yes,yes,i don't know,somewhat difficult,...,rarely,sometimes,52,male,united states of america,georgia,united states of america,georgia,back-end developer,sometimes
1431,0,100-500,0.0,1.0,i don't know,i am not sure,no,yes,i don't know,somewhat difficult,...,sometimes,often,30,female,united states of america,nebraska,united states of america,nebraska,devops/sysadmin,sometimes
